In [ ]:
import gdown
import pandas as pd
import numpy as np
import os
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import logging
from datetime import date, timedelta
import datetime
import re

In [ ]:
url = "https://drive.google.com/drive/folders/1tq2wthWD_eFpuT_cJwLs4rxDs0h1OfBd"
gdown.download_folder(url, quiet=True, use_cookies=False)

['/content/data_quality_rk_clss/book_master_2023-03-24.csv']

In [ ]:
#https://medium.com/analytics-vidhya/how-to-connect-google-drive-to-python-using-pydrive-9681b2a14f20
# https://pythonhosted.org/PyDrive/oauth.html#authentication-in-two-lines
# gauth = GoogleAuth()
# Create local webserver and auto handles authentication.
# gauth.LocalWebserverAuth()

# **** Data Transformation Part ****

In [ ]:
# https://www.w3schools.com/python/python_datetime.asp
today = date.today().strftime("%Y-%m-%d") #2022-10-29
yesterday = date.today()- datetime.timedelta(days=1)
yesterday = yesterday.strftime("%Y-%m-%d")
designatedFileName = 'book_master_' + yesterday+'.csv' # to fix this
# designatedFileName='book_master_2023-03-24.csv' # hardcode
print(designatedFileName)

book_master_2023-06-06.csv


In [ ]:
today #ทำไมต้องใช้ yesterday เพราะตกลงกับลูกค้าไว้ว่าถ้าลูกค้าเอาข้อมูลมาวางเมื่อไหร่วันต่อไปจะ run ให้

'2023-06-07'

In [ ]:
# https://docs.python.org/3/library/logging.html
log_file_name=today+'_log.log'

logging.basicConfig(filename=log_file_name,
                    format='%(asctime)s %(message)s',
                    filemode='w',level=logging.INFO)

logger = logging.getLogger()

In [ ]:
df=pd.read_csv(f'data_quality_rk_clss/{designatedFileName}') # จะErrorต้องเปลี่ยน date ถ้าจะกลับมาทำอีก
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN


In [ ]:
# file_list = drive.ListFile({'q': "'1tq2wthWD_eFpuT_cJwLs4rxDs0h1OfBd' in parents and trashed=false"}).GetList()
# file_df=pd.DataFrame(file_list)

#Check if the file is existed

# file_df[file_df.title==designatedFileName]['id'].values[0]

# if(file_df[file_df.title==designatedFileName].shape[0]!=0):
#     print(f'{designatedFileName}` detected')
#     logger.info(f'`{designatedFileName}` detected')
# else:
#     logger.error(f'`{designatedFileName}` was not found')
#     raise ImportError(f'`{designatedFileName}` was not found')

In [ ]:
# # https://pythonhosted.org/PyDrive/filemanagement.html#download-file-content

# designatedID=file_df[file_df.title==designatedFileName]['id'].values[0]
# tmpFileName='tmp_'+designatedFileName
# drive.CreateFile({'id': designatedID}).GetContentFile(tmpFileName)
# df=pd.read_csv(tmpFileName)

## 1. REMOVE QUANTITY <=0

In [ ]:
#create df that has no quantity<=0
df[df.Quantity == 0] # select *

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
892,3892,The Course of Love,Fiction,Three,16.78,In stock,0,M.Bridges,TOR,2022-09-25,NaN
895,3895,"Run, Spot, Run: The Ethics of Keeping Pets",Philosophy,One,20.02,In stock,0,M.Bridges,TOR,2022-09-25,NaN
896,3896,New Moon (Twilight #2),Young Adult,Four,12.86,In stock,0,E. Barron,TOR,2022-09-25,NaN
898,3898,Kindle Paperwhite User's Guide,Nonfiction,Three,34.00,In stock,0,A. Edwards,Dragonsteel,2022-09-25,NaN
899,3899,H is for Hawk,Nonfiction,Five,57.42,In stock,0,A. Edwards,O' Reilly,2022-09-25,NaN


In [ ]:
df[["Title"]][df.Quantity == 0] #ต่างจากด้านบนคือ [[]] -> ดึงเเค่ column ที่สนใจ ## select Title

,Title
892,The Course of Love
895,"Run, Spot, Run: The Ethics of Keeping Pets"
896,New Moon (Twilight #2)
898,Kindle Paperwhite User's Guide
899,H is for Hawk


In [ ]:
# Remove Quantity <=0
df = df[df.Quantity > 0]

In [ ]:
df[["Title"]][df.Quantity == 0]

,Title


## 2. QC book title can't have multiple rows

In [ ]:
#group by book title and count row
df.groupby('Title').count()
g = df.groupby('Title').count()
g

,book_id,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
Title,,,,,,,,,,
"""Most Blessed of the Patriarchs"": Thomas Jefferson and the Empire of the Imagination",1,1,1,1,1,1,1,1,1,0
#GIRLBOSS,1,1,1,1,1,1,1,1,1,0
#HigherSelfie: Wake Up Your Life. Free Your Soul. Find Your Tribe.,1,1,1,1,1,1,1,1,1,0
'Salem's Lot,1,1,1,1,1,1,1,1,1,0
(Un)Qualified: How God Uses Broken People to Do Big Things,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...
Zealot: The Life and Times of Jesus of Nazareth,1,1,1,1,1,1,1,1,1,0
Zero History (Blue Ant #3),1,1,1,1,1,1,1,1,1,0
"Zero to One: Notes on Startups, or How to Build the Future",1,1,1,1,1,1,1,1,1,0


In [ ]:
g[g.Book_category > 1 ]
#g[g['Book_category' ] > 1 ]

,book_id,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
Title,,,,,,,,,,
America's War for the Greater Middle East: A Military History,2,2,2,2,2,2,2,2,2,0
The Star-Touched Queen,2,2,2,2,2,2,2,2,2,0


In [ ]:
g[g['Book_category' ] > 1 ].index

Index(['America's War for the Greater Middle East: A Military History', 'The Star-Touched Queen'], dtype='object', name='Title')

In [ ]:
#add dup title to dup_title
dup_title = g[g['Book_category' ] > 1 ].index.tolist()
dup_title

["America's War for the Greater Middle East: A Military History",
 'The Star-Touched Queen']

In [ ]:
df[df.Title.isin(dup_title)]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
236,3236,The Star-Touched Queen,Fantasy,Five,46.02,In stock,14,E. Barron,O' Reilly,2022-09-25,NaN
358,3358,The Star-Touched Queen,Fantasy,Five,46.00,In stock,14,E. Bledsoe,O'Reilly,2022-09-25,NaN
639,3639,America's War for the Greater Middle East: A M...,History,Two,51.22,In stock,4,M.Bridges,Dragonsteel,2022-09-24,NaN
640,3640,America's War for the Greater Middle East: A M...,History,Two,51.22,In stock,5,C.Payne,Dragonsteel,2022-09-25,NaN


In [ ]:
# วิธีนี้คือการลบ dup_title ออกเลย
#df[~df.Title.isin(dup_title)]

In [ ]:
#remove title which has multiple rows (by randomly)
df = df.drop_duplicates( subset = ['Title'] )

In [ ]:
df[df.Title.isin(dup_title)]


,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
236,3236,The Star-Touched Queen,Fantasy,Five,46.02,In stock,14,E. Barron,O' Reilly,2022-09-25,NaN
639,3639,America's War for the Greater Middle East: A M...,History,Two,51.22,In stock,4,M.Bridges,Dragonsteel,2022-09-24,NaN


## 3. Rename Thai column

In [ ]:
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN


In [ ]:
#rename สำนักพิมพ์ to Publisher
df.rename(columns = {'สำนักพิมพ์' : 'Publisher' } , inplace = True)
#df = df.rename(columns = {'สำนักพิมพ์' : 'Publisher'} )
df.tail(3)

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date
998,3998,A Spy's Devotion (The Regency Spies of London #1),Historical Fiction,Five,16.97,In stock,1,A. Edwards,Dragonsteel,2022-09-25,NaN
999,3999,1st to Die (Women's Murder Club #1),Mystery,One,53.98,In stock,1,A. Edwards,O' Reilly,2022-09-25,NaN
1000,4000,"1,000 Places to See Before You Die",Travel,Five,26.08,In stock,1,E. Barron,O' Reilly,2022-09-25,NaN


## 4. Amend Thai value inside the cell to English

In [ ]:
df.Book_category.value_counts()

Default               152
Nonfiction            108
Sequential Art         75
Add a comment          67
Fiction                64
Young Adult            48
Fantasy                47
Romance                35
Mystery                32
Food and Drink         30
Childrens              29
Historical Fiction     26
Classics               19
Poetry                 18
History                18
Horror                 17
Womens Fiction         17
Science Fiction        16
Science                14
Music                  13
Business               12
Thriller               11
Travel                 11
Humor                  10
Philosophy             10
Autobiography           9
Art                     8
Psychology              7
Religion                7
Spirituality            6
Christian Fiction       6
New Adult               6
Sports and Games        5
YA                      5
Biography               5
Self Help               5
Health                  4
Contemporary            3
Politics    

In [ ]:
df.loc[df.Book_category=='กวี']
#df[df.Book_category=='กวี']  ## .loc ไม่จำเป็นต้องมีก็ได้

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date
40,3040,Slow States of Collapse: Poems,กวี,Three,57.31,In stock,17,A. Edwards,Dragonsteel,2022-09-25,NaN


In [ ]:
#change row that has [กวี] value to [Poetry] value
df['Book_category'] = np.where(df.Book_category == 'กวี', 'Poetry', df.Book_category)
                            #np.where(เงื่อนไข , เป็น , else ถ้าไม่ใช่ใช้ตัวดั้งเดิม ) # where -> if+else


In [ ]:
#check result
df.loc[df.book_id==3040]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,Publisher,PublishedDate,add_date
40,3040,Slow States of Collapse: Poems,Poetry,Three,57.31,In stock,17,A. Edwards,Dragonsteel,2022-09-25,NaN


## 5. Create column: [import_date] = [today] value


In [ ]:
# set import_date = today
df xxxxx
df.tail()

In [ ]:
df['import_date'].unique()

## 6. check date column to [YYYY-MM-DD] date format DD/MM/YYYY e.g. 25/09/2022 to 2022-09-25



In [ ]:
#see this doc: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-format-codes

In [ ]:
df.loc[df.book_id.isin([3665,3040])]

,book_id,Title,Book_category,Star_rating,Price,Stock,Quantity,user_input,สำนักพิมพ์,PublishedDate,add_date
665,3665,No One Here Gets Out Alive,Music,Five,20.02,In stock,4,E. Barron,TOR,25/09/2022,NaN


In [ ]:
#reformat [PublishedDate] with new column name: [PublishedDate_cleaned]
df['PublishedDate_cleaned']=xxxx

In [ ]:
df[['PublishedDate','PublishedDate_cleaned']].value_counts()

In [ ]:
# Drop column:[PublishedDate]
df.drop(xxxxx)
# Rename [PublishedDate_cleaned] to [PublishedDate]
df.rename(xxxxx,inplace=True)

In [ ]:
df.tail(3)

## 7. Select certain columns

In [ ]:
df.columns

In [ ]:
df=df[['book_id',
 'Title',
 'Book_category',
 'Star_rating',
 'Price',
 'Stock',
 'Quantity',
 'Publisher','PublishedDate',xxxx]]

In [ ]:
df.tail(3)

 ### ----- Transformation Part Completed -----

# **** Data Quality Part ****

## 1.Check template column. Must be the same

In [ ]:
template_col=['book_id','Title','Book_category','Star_rating','Price','Stock','Quantity','Publisher','PublishedDate','import_date']

In [ ]:
if ((template_col==xxxxx))==False:

    logger.error("File columns don't match the current table format")
    logger.error(f"Unsuccessful:File columns don't match the current table format \n template: {template_col} \n csv file: {df.columns} ")
    raise ValueError(f"Unsuccessful:File columns don't match the current table format \n template: {template_col} \n csv file: {df.columns} ")

else:
    print('File columns match the current table format')
    logger.info('File columns match the current table format')

1.1 Check if there is quantity <=0

In [ ]:
df[df.Quantity<=0]

In [ ]:
if (xxxx)>1):
    raise ImportError(f'`{df}` quantity has <=0')
else:
    print('passs')

##  2. book title can't have multiple rows

In [ ]:
g=df.groupby('Title').count()
g
if(len(g[g.Book_category>1]>1)):
    err_txt='Book_category>1 per title'
    logger.error(err_txt)
    raise ImportError(err_txt)
else:
    print('pass')

## 3. If import_date = today

In [ ]:
pd.Series(df.import_date==today).all()==True

AttributeError: ignored

In [ ]:
if(xxxxx==False):

    raise ImportError('update date is not today')
else:
    print('pass')

## -------------- END --------------

## Data Transformation + DQC were done, then what??